In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.typing as npt

from scipy.optimize import curve_fit

from stellar_stream import StellarStream

from scipy.stats import pearsonr
import pandas as pd
import warnings



In [ ]:
### Load all simulations into StellarStream type and put in a dictionary for easy access.

from pathlib import Path
p = Path('../../data/sample')
graph = []

simulations = {}

for sim_data in p.glob('*.npz'):
    simulation_number = sim_data.name.removeprefix('simdatag').removesuffix('.npz')
    simulations[simulation_number] = (StellarStream.from_simulation(simulation_number, sim_data)
                                    #  .select("restrict_phi1", phi1_lim=(-110, 10))
                                    #  .select("restrict_phi2", phi2_lim=10.0)
                                    #  .run_analysis(name="detrend", polynomial_fit_degree=4)
                                    .run_analysis(name="gaussian_process_detrend", n_bins=40)
                                     .select("restrict_phi1", phi1_lim=(-100, 0))
                                     .select("restrict_phi2", phi2_lim=3.0)
    )

In [ ]:
for sim in simulations.values():
    sim.plot_power_spectrum(smooth=True, sigma=2, interpolate=True)

In [ ]:
simulations['582'].plot_stream()
simulations['582'].plot_density(smooth=True, sigma=2)
simulations['582'].plot_power_spectrum(smooth=True, sigma=2, interpolate=True)

## How sensitive is dynamical heating measure to preprocessing chain?

In [ ]:
p = Path('../Data')

df = pd.DataFrame()

heat_1 = {}
heat_2 = {}
heat_3 = {}

for sim_data in p.glob('*.npz'):
    simulation_number = sim_data.name.removeprefix('simdatag').removesuffix('.npz')
    heat_1[simulation_number] = (StellarStream.from_simulation(simulation_number, sim_data)
                                      .select("restrict_phi1", phi1_lim=(-100, 0))
                                      .select("restrict_phi2", phi2_lim=3.0)
                                      .run_analysis(name="detrend", polynomial_fit_degree=4)
                                      .vlos.std(ddof=1))
    
    heat_2[simulation_number] = (StellarStream.from_simulation(simulation_number, sim_data)
                                     .select("restrict_phi1", phi1_lim=(-110, 10))
                                     .select("restrict_phi2", phi2_lim=10.0)
                                     .run_analysis(name="detrend", polynomial_fit_degree=4)
                                     .select("restrict_phi1", phi1_lim=(-100, 0))
                                     .select("restrict_phi2", phi2_lim=3.0)
                                     .vlos.std(ddof=1))
    heat_3[simulation_number] = (StellarStream.from_simulation(simulation_number, sim_data)
                                     .select("restrict_phi1", phi1_lim=(-100, 0))
                                     .run_analysis(name="detrend", polynomial_fit_degree=4)
                                     .select("restrict_phi2", phi2_lim=3.0)
                                     .vlos.std(ddof=1))

    df[simulation_number] = pd.Series({
        "heat_1": heat_1[simulation_number],
        "heat_2": heat_2[simulation_number],
        "heat_3": heat_3[simulation_number]
    })



In [ ]:
from scipy.stats import skew
df.sort_index(axis=1, inplace=True)
df.T.plot()

## Using Skew to characterize wether a simulation still has a trend

In [ ]:
S = StellarStream.from_simulation('716', '../Data/simdatag718.npz')
S = (S
    # .select("restrict_phi1", phi1_lim=(-110, 10))
    # .select("restrict_phi2", phi2_lim=10.0)
    # .run_analysis(name="detrend", polynomial_fit_degree=4)
    .select("restrict_phi1", phi1_lim=(-100, 0))
    .select("restrict_phi2", phi2_lim=3.0))

print(np.polyfit(S.phi1, S.phi2, 3))

S.plot_stream()

print(f'skew phi1: {skew(S.phi1)}')

In [ ]:
df_skew = pd.DataFrame()
for key in simulations.keys():
    df_skew[key] = pd.Series({
    "skew": skew(simulations[key].phi1)
    })

df_skew.sort_index(axis=1,inplace=True)
df_skew = df_skew.T

In [ ]:
df_skew

### B-Spline Fitting

In [ ]:
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import make_lsq_spline, UnivariateSpline

S = StellarStream.from_simulation('716', '../Data/simdatag718.npz')
S = (S
    .select("restrict_phi1", phi1_lim=(-100, 0))
    .select("restrict_phi2", phi2_lim=3.0))


def fit_bspline(phi1, phi2, k=3, n_internal_knots=4, jitter_duplicate_x=False):
    """
    Fit a least-squares B-spline to (phi1, phi2).

    Parameters
    ----------
    phi1, phi2 : array-like, same shape
        Independent and dependent data.
    k : int
        Spline degree (default 3 = cubic).
    n_internal_knots : int
        Number of internal knots to place (evenly spaced) inside (min(phi1), max(phi1)).
        If 0, a global polynomial spline of degree k is fitted.
    jitter_duplicate_x : bool
        If True, tiny jitter is added to duplicated phi1 values to avoid linear algebra errors.
        If False, duplicated phi1 values are averaged (recommended if duplicates are real repeats).

    Returns
    -------
    spl : BSpline-like object (callable)
        The fitted spline. Call with spl(x) to evaluate.
    trend_phi2 : ndarray
        spline evaluated at original phi1 (unsorted, same order as inputs).
    residual_phi2 : ndarray
        phi2 - trend_phi2
    """
    phi1 = np.asarray(phi1)
    phi2 = np.asarray(phi2)
    if phi1.shape != phi2.shape:
        raise ValueError("phi1 and phi2 must have the same shape")

    # sort by phi1 because make_lsq_spline expects x increasing
    order = np.argsort(phi1)
    x = phi1[order].astype(float)
    y = phi2[order].astype(float)

    # handle duplicates: average y for duplicate x (common approach)
    uniq_x, inv_idx = np.unique(x, return_inverse=True)
    if len(uniq_x) < len(x):
        # average y for duplicate x
        yy = np.zeros_like(uniq_x)
        counts = np.zeros_like(uniq_x)
        for i,u in enumerate(inv_idx):
            yy[u] += y[i]
            counts[u] += 1
        yy /= counts
        x = uniq_x
        y = yy

    if jitter_duplicate_x:
        # add tiny jitter if duplicates still present (not usually necessary after averaging)
        diffs = np.diff(x)
        if np.any(diffs == 0):
            x = x + np.random.normal(scale=1e-12 * np.ptp(x), size=x.shape)

    # build internal knot vector (evenly spaced inside the open interval)
    if n_internal_knots > 0:
        t_internal = np.linspace(x[0], x[-1], n_internal_knots + 2)[1:-1]
    else:
        t_internal = np.asarray([])

    # check that we have more data points than spline coefficients
    # number of basis functions (coeffs) = len(t_internal) + k + 1
    n_coeffs = len(t_internal) + k + 1
    if x.size <= n_coeffs:
        raise ValueError(
            f"Not enough data points ({x.size}) for the requested spline complexity "
            f"(need > {n_coeffs} coefficients = n_internal_knots({len(t_internal)}) + k({k}) + 1). "
            "Reduce n_internal_knots or degree k, or use more data."
        )

    # attempt LSQ-spline
    try:
        spl = make_lsq_spline(x, y, t_internal, k=k)
    except Exception as e:
        # fall back to a smoothing spline if LSQ-spline fails (gives a graceful alternative)
        # choose an automatic smoothing factor: roughly variance * n_points
        s = np.var(y) * max(1.0, x.size * 1e-3)
        spl = UnivariateSpline(x, y, k=k, s=s)
        # note: UnivariateSpline returns a similar callable interface

    # evaluate at original (unsorted) phi1 order
    trend = spl(phi1)    # spl accepts array-like
    residual = phi2 - trend
    return spl, trend, residual



spl, trend_phi2, residual_phi2 = fit_bspline(S.phi1, S.phi2, k=1, n_internal_knots=2)

# ensure we plot in increasing phi1 order so points/lines line up
order_phi2 = np.argsort(S.phi1.values if hasattr(S.phi1, "values") else S.phi1)
x_sorted = (S.phi1 if hasattr(S.phi1, "values") else S.phi1)[order_phi2]
y_sorted = (S.phi2 if hasattr(S.phi2, "values") else S.phi2)[order_phi2]
trend_sorted = trend_phi2[order_phi2]
resid_sorted = residual_phi2[order_phi2]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))

# left: data + trend (also add a smooth spline curve for visual clarity)
ax1.scatter(x_sorted, y_sorted, s=1, label='data')
ax1.plot(x_sorted, trend_sorted, lw=1.5, label='spline trend', color='C1')
# optional: smooth curve from the spline on a dense grid
xx = np.linspace(x_sorted.min(), x_sorted.max(), 500)
ax1.plot(xx, spl(xx), lw=1, linestyle='--', label='spline (dense grid)', alpha=0.7)
ax1.set_title('phi2 and trend')
ax1.set_xlabel('phi1')
ax1.set_ylabel('phi2')
ax1.legend()
ax1.grid(True, alpha=0.2)

# right: residuals
ax2.scatter(x_sorted, resid_sorted, s=1)
ax2.axhline(0, ls='--', lw=1, color='k')
ax2.set_title('residual phi2 (data - trend)')
ax2.set_xlabel('phi1')
ax2.set_ylabel('residual')
ax2.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()


### Comparison of Density

In [ ]:
(S
    .select("restrict_phi1", phi1_lim=(-100, 0))
    .select("restrict_phi2", phi2_lim=6.0)
    .plot_power_spectrum())

(S
    .select("restrict_phi1", phi1_lim=(-110, 10))
    .select("restrict_phi2", phi2_lim=15.0)
    .run_analysis(name="detrend", polynomial_fit_degree=4)
    .select("restrict_phi1", phi1_lim=(-100, 0))
    .select("restrict_phi2", phi2_lim=6.0)
    .plot_power_spectrum())

(S
    .select("restrict_phi1", phi1_lim=(-110, 10))
    .select("restrict_phi2", phi2_lim=15.0)
    .run_analysis(name="gaussian_detrend", smoothing_scale=0.001)
    .select("restrict_phi1", phi1_lim=(-100, 0))
    .select("restrict_phi2", phi2_lim=6.0)
    .plot_power_spectrum())

In [ ]:
S = StellarStream.from_simulation('656', '../Data/simdatag656.npz')

(S
    .select("restrict_phi1", phi1_lim=(-100, 0))
    .select("restrict_phi2", phi2_lim=3.0)
    .run_analysis(name="detrend", polynomial_fit_degree=4)
    .plot_density())

(S
    .run_analysis(name="detrend", polynomial_fit_degree=4)
    .select("restrict_phi1", phi1_lim=(-100, 0))
    .select("restrict_phi2", phi2_lim=3.0)
    .plot_density())

(S
    .select("restrict_phi1", phi1_lim=(-100, 0))
    .run_analysis(name="detrend", polynomial_fit_degree=4)
    .select("restrict_phi2", phi2_lim=3.0)
    .plot_density())




### Gaussian Filter Detrend

In [ ]:
S = StellarStream.from_simulation('584', '../Data/simdatag584.npz')
# S = (S
#     .run_analysis(name="detrend", polynomial_fit_degree=4)
#     .select("restrict_phi1", phi1_lim=(-100, 0))
#     .select("restrict_phi2", phi2_lim=3.0)
#     .plot_density())

# convert to numpy arrays for computation but keep originals for re-wrapping
phi1_orig = S.phi1
phi2_orig = S.phi2
vlos_orig = S.vlos

x = np.asarray(phi1_orig, dtype=float)
y_phi2 = np.asarray(phi2_orig, dtype=float)
y_vlos = np.asarray(vlos_orig, dtype=float)

# sort
order_phi2 = np.argsort(x)
x_sorted = x[order_phi2]
phi2_sorted = y_phi2[order_phi2]
vlos_sorted = y_vlos[order_phi2]

# compute representative dx (ignore zero diffs from duplicates)
diffs = np.diff(x_sorted)
nonzero = diffs[np.abs(diffs) > 0]
if nonzero.size == 0:
    raise ValueError("phi1 has no spacing (all values equal). Cannot compute smoothing in pixel units.")
dx = np.median(nonzero)

smoothing_scale=0.5
# convert smoothing scale (same units as phi1) to pixels for gaussian_filter1d
sigma_pix = float(smoothing_scale) / float(dx)
if sigma_pix <= 0:
    # sigma=0 means no smoothing; gaussian_filter1d with sigma=0 returns original array
    sigma_pix = 0.0

# apply Gaussian smoothing on the sorted data (reflect mode to reduce edge artifacts)
# gaussian_filter1d accepts sigma==0 (no smoothing)
phi2_trend_sorted = gaussian_filter1d(phi2_sorted, sigma_pix, mode='reflect')
vlos_trend_sorted = gaussian_filter1d(vlos_sorted, sigma_pix, mode='reflect')

# map the trends back to original order
inverse_order = np.argsort(order_phi2)
phi2_trend_orig = phi2_trend_sorted[inverse_order]
vlos_trend_orig = vlos_trend_sorted[inverse_order]

# detrend in original order
phi2_detrend = np.asarray(phi2_orig, dtype=float) - phi2_trend_orig
vlos_detrend = np.asarray(vlos_orig, dtype=float) - vlos_trend_orig

fig, ax = plt.subplots(1, 2, figsize=(20, 5))

fig.suptitle(f'Stellar Stream Gaussian Detrend with {smoothing_scale} Pixel Smoothing')

ax[0].scatter(x=phi1_orig, y=phi2_orig, s=0.1, label='Original')
ax[0].scatter(x=phi1_orig, y=phi2_trend_orig, s=0.1, label='Trend')
ax[0].legend()

ax[1].scatter(x=phi1_orig, y=phi2_detrend, s=0.1, label='Detrended')
ax[1].legend()
ax[1].axhline(0, ls='--', lw=1, color='k')

ax[0].set_xlabel(r'$\phi_1$')
ax[1].set_xlabel(r'$\phi_1$')
ax[0].set_ylabel(r'$\phi_2$')
ax[1].set_ylabel(r'$\phi_2$ Detrended')

In [ ]:
from scipy.optimize import minimize
from celerite2 import GaussianProcess, terms
import time

S = StellarStream.from_simulation('580', '../Data/simdatag580.npz')

def bin_data(x, y, n_bins=2000):
    # x sorted assumed
    edges = np.linspace(x.min(), x.max(), n_bins + 1)
    idx = np.digitize(x, edges) - 1
    idx[idx < 0] = 0
    idx[idx >= n_bins] = n_bins - 1
    xb = 0.5 * (edges[:-1] + edges[1:])
    # compute mean y and std per bin (ignoring empty bins)
    ysum = np.bincount(idx, weights=y, minlength=n_bins)
    count = np.bincount(idx, minlength=n_bins)
    with np.errstate(invalid='ignore'):
        ymean = ysum / count
    mask = count > 0
    return xb[mask], ymean[mask], count[mask]

# prepare sorted data
phi1 = np.asarray(S.phi1, dtype=float)
phi2 = np.asarray(S.phi2, dtype=float)
mask_phi2 = np.isfinite(phi1) & np.isfinite(phi2)
phi1 = phi1[mask_phi2]; phi2 = phi2[mask_phi2]
order_phi2 = np.argsort(phi1)
phi1 = phi1[order_phi2]; phi2 = phi2[order_phi2]

# bin: reduce to ~2000
n_bins = 30
xb, yb, counts = bin_data(phi1, phi2, n_bins=n_bins)

# estimate uncertainties for binned points (std/sqrt(N) or use robust estimate)
yerr_binned = np.sqrt(np.maximum(np.var(yb), 1e-6)) / np.sqrt(np.maximum(counts, 1))

# auto initial guesses (as we used before)
amp0 = max(np.std(phi2), 1e-6)
stream_length = phi1.max() - phi1.min() if (phi1.max() > phi1.min()) else 1.0
dx_med = np.median(np.diff(phi1))
rho0 = max(stream_length / 8.0, 3.0 * dx_med, 1e-3)
jitter0 = max(1e-3 * amp0, 1e-6)
print("bin-fit: amp0, rho0, jitter0:", amp0, rho0, jitter0)

# Build celerite2 GP for binned data
term = terms.SHOTerm(sigma=amp0, rho=rho0, Q=1.0)   # or try Matern32Term
gp = GaussianProcess(term, mean=np.median(yb))
gp.compute(xb, diag=(yerr_binned**2 + jitter0))

# Fit only (log sigma, log rho, log jitter) - keep mean fixed
def set_params(params, gp_local):
    sigma = np.exp(params[0])
    rho = np.exp(params[1])
    jitter = np.exp(params[2])
    gp_local.mean = np.median(yb)            # fixed mean
    gp_local.kernel = terms.SHOTerm(sigma=sigma, rho=rho, Q=1.0)
    gp_local.compute(xb, diag=(yerr_binned**2 + jitter), quiet=True)
    return gp_local

def neg_log_like(params):
    g = set_params(params, gp)
    return -g.log_likelihood(yb)

p0 = np.array([np.log(amp0), np.log(rho0), np.log(jitter0)])
t0 = time.time()
sol = minimize(neg_log_like, p0, method="L-BFGS-B", options={'maxiter':200})
t_fit = time.time() - t0
print("Fit done: success", sol.success, "msg:", sol.message, "time(s):", t_fit)

# set gp with fitted params
gp = set_params(sol.x, gp)

# Predict on full dataset (this is linear-time and quick)
t0 = time.time()
mu_full, var_full = gp.predict(yb, t=phi1, return_var=True)  # note: predict expects training y input; for celerite2 we pass yb (binned y) used for fit
# However celerite2.predict requires you to pass the training y used when compute was called.
# If you used the binned data to fit, pass yb for prediction calls; gp.compute was called with xb.
t_pred = time.time() - t0
print("Predict on full N time(s):", t_pred)

residuals = phi2 - mu_full


# GP prediction at full resolution
mu_full, var_full = gp.predict(yb, t=phi1, return_var=True)
residuals = phi2 - mu_full

# Plot
plt.figure(figsize=(20,5))

plt.subplot(121)
plt.scatter(phi1, phi2, s=1, alpha=0.5, label="data")
plt.plot(phi1, mu_full, color="C1", lw=2, label="GP trend")
plt.fill_between(phi1, mu_full-np.sqrt(var_full), mu_full+np.sqrt(var_full),
                 color="C1", alpha=0.3, label="GP ±1σ")
plt.legend()
plt.title("Trend fit with celerite2")

plt.subplot(122)
plt.scatter(phi1, residuals, s=1, alpha=0.5, color="k")
plt.axhline(0, color="C1", lw=2)
plt.title("Residuals")

plt.show()

In [ ]:

phi1_phi2_fit = np.polynomial.polynomial.Polynomial.fit(S.phi1, S.phi2, 4)
phi2_detrend = S.phi2 - phi1_phi2_fit(S.phi1)


plt.figure(figsize=(20,4))
plt.subplot(1,2,1)
plt.scatter(S.phi1, S.phi2, s=6, alpha=0.6, label='data')
plt.plot(phi1_grid, phi1_phi2_fit(phi1_grid), color='C1', lw=2, label='celerite2 trend')
plt.xlabel('phi1'); plt.ylabel('phi2'); plt.legend()

plt.subplot(1,2,2)
plt.scatter(S.phi1, phi2_detrend, s=6)
plt.axhline(0, color='k', ls='--')
plt.xlabel('phi1'); plt.ylabel('residual (phi2 - trend)')
plt.tight_layout()
plt.show()

In [ ]:


S = StellarStream.from_simulation('580', '../Data/simdatag580.npz')



def gaussian_process_detrend(self, n_bins):
    def bin_data(x, y, n_bins=2000):
        # x sorted assumed
        edges = np.linspace(x.min(), x.max(), n_bins + 1)
        idx = np.digitize(x, edges) - 1
        idx[idx < 0] = 0
        idx[idx >= n_bins] = n_bins - 1
        xb = 0.5 * (edges[:-1] + edges[1:])
        # compute mean y and std per bin (ignoring empty bins)
        ysum = np.bincount(idx, weights=y, minlength=n_bins)
        count = np.bincount(idx, minlength=n_bins)
        with np.errstate(invalid='ignore'):
            ymean = ysum / count
        mask = count > 0
        return xb[mask], ymean[mask], count[mask]

    # prepare sorted data
    phi1 = np.asarray(self.phi1, dtype=float)
    phi2 = np.asarray(self.phi2, dtype=float)
    vlos = np.asarray(self.vlos, dtype=float)
    mask_phi2 = np.isfinite(phi1) & np.isfinite(phi2) & np.isfinite(vlos)
    phi1 = phi1[mask_phi2]; phi2 = phi2[mask_phi2]; vlos = vlos[mask_phi2]
    order_phi2 = np.argsort(phi1)
    phi1 = phi1[order_phi2]; phi2 = phi2[order_phi2]

    # bin: reduce to ~2000
    xb, yb, counts = bin_data(phi1, phi2, n_bins=n_bins)

    # estimate uncertainties for binned points (std/sqrt(N) or use robust estimate)
    yerr_binned = np.sqrt(np.maximum(np.var(yb), 1e-6)) / np.sqrt(np.maximum(counts, 1))

    # auto initial guesses (as we used before)
    amp0 = max(np.std(phi2), 1e-6)
    stream_length = phi1.max() - phi1.min() if (phi1.max() > phi1.min()) else 1.0
    dx_med = np.median(np.diff(phi1))
    rho0 = max(stream_length / 8.0, 3.0 * dx_med, 1e-3)
    jitter0 = max(1e-3 * amp0, 1e-6)

    # Build celerite2 GP for binned data
    term = terms.SHOTerm(sigma=amp0, rho=rho0, Q=1.0)   # or try Matern32Term
    gp = GaussianProcess(term, mean=np.median(yb))
    gp.compute(xb, diag=(yerr_binned**2 + jitter0))

    # Fit only (log sigma, log rho, log jitter) - keep mean fixed
    def set_params(params, gp_local):
        sigma = np.exp(params[0])
        rho = np.exp(params[1])
        jitter = np.exp(params[2])
        gp_local.mean = np.median(yb)            # fixed mean
        gp_local.kernel = terms.SHOTerm(sigma=sigma, rho=rho, Q=1.0)
        gp_local.compute(xb, diag=(yerr_binned**2 + jitter), quiet=True)
        return gp_local

    def neg_log_like(params):
        g = set_params(params, gp)
        return -g.log_likelihood(yb)

    p0 = np.array([np.log(amp0), np.log(rho0), np.log(jitter0)])
    sol = minimize(neg_log_like, p0, method="L-BFGS-B", options={'maxiter':200})

    # set gp with fitted params
    gp = set_params(sol.x, gp)

    # GP prediction at full resolution
    mu_full, var_full = gp.predict(yb, t=phi1, return_var=True)
    phi2_detrend = phi2 - mu_full
    
    return StellarStream(self.phi1, phi2_detrend, vlos_detrend, name=self.name)
